# Working notebook to get the Reference Point Reversal script working without Matlab

## Setup

In [31]:
import pydicom
import os
import sys
import time
import copy
import inspect

In [51]:
sourceFile = os.path.join(os.getcwd(), 'RP.dcm')
saveFile = sourceFile.replace('.dcm', '-Reversed.dcm')
filename = os.path.basename(sourceFile)
path = os.path.dirname(sourceFile)
referencePlan = pydicom.dcmread(sourceFile)
# Create a copy of the original file
flippedPlan = copy.deepcopy(referencePlan)
# Modify the copy name to indicate it's been reversed
flippedPlan.RTPlanLabel = referencePlan.RTPlanLabel + "-Reversed"
# Plan name safety, if the plan name is too long it will be cut
if (len(flippedPlan.RTPlanLabel) > 16):
    flippedPlan.RTPlanLabel = referencePlan.RTPlanLabel[0:11] + "-Rev"



## Testing

In [52]:
print("Modifying the file...")
print(sourceFile)
print(filename)
print(path)
print(referencePlan.RTPlanLabel)
print(flippedPlan.RTPlanLabel)

Modifying the file...
h:\Source\VMAT-TBI-crcrewso\Matlab\RP.dcm
RP.dcm
h:\Source\VMAT-TBI-crcrewso\Matlab
P1 HN 06x
P1 HN 06x-Rev


In [55]:
print(referencePlan)

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 232
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: RT Plan Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.2.826.0.1.3680043.9.4245.6163269327563821045967741266632670434
(0002, 0010) Transfer Syntax UID                 UI: Implicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.826.0.1.3680043.9.4245.0.0.0.0.2.7.1
(0002, 0013) Implementation Version Name         SH: 'ALIZA2.7.1'
(0002, 0016) Source Application Entity Title     AE: 'ALIZAAE'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 192'
(0008, 0012) Instance Creation Date              DA: '20240818'
(0008, 0013) Instance Creation Time              TM: '134706'
(0008, 0016) SOP Class UID                       UI: RT Plan Storage
(0008, 0018) SOP Instance UI

## Algorithm

In [54]:
for i in range(len(referencePlan.BeamSequence)):
    CPcount = len(referencePlan.BeamSequence[i].ControlPointSequence)
    CPList = list(referencePlan.BeamSequence[i].ControlPointSequence)
    ControlPoints = referencePlan.BeamSequence[i].ControlPointSequence
    for j in range(CPcount):
        if j == 0:
            Col = ControlPoints[CPList[j]].BeamLimitingDeviceAngle
            if Col > 180:
                Col = Col - 180
            else:
                Col = Col + 180
            flippedPlan.BeamSequence[i].ControlPointSequence[CPList[j]].BeamLimitingDeviceAngle = Col
        Dir = ControlPoints[CPList[j]].GantryRotationDirection
        if Dir == 'CW':
            flippedPlan.BeamSequence[i].ControlPointSequence[CPList[j]].GantryRotationDirection = 'CC'
        elif Dir == 'CC':
            flippedPlan.BeamSequence[i].ControlPointSequence[CPList[j]].GantryRotationDirection = 'CW'
        elif Dir == 'NONE':
            print('Done')
            print(i)
        else:
            raise ValueError('No Gantry')
        swapAngle = ControlPoints[CPList[CPcount-j-1]].GantryAngle
        flippedPlan.BeamSequence[i].ControlPointSequence[CPList[j]].GantryAngle = swapAngle
flippedPlan.save_as(os.path.join(path, saveFile))


TypeError: list indices must be integers or slices, not Dataset